### anchor-fluvial-metrics

`analog_image_generator.stats.compute_metrics(gray, masks, env)` composes Phase 1/2 outputs (variograms, PSD, topology, stack metadata) referenced by PRD + reporting.


### anchor-fluvial-variogram

`compute_variogram` + `fit_power_law` and `two_segment_fit` derive β_iso, β_dir, β_seg1/β_seg2, and `h0` used in CSV/PDF deliverables.


### anchor-fluvial-psd-topology

`psd_anisotropy`, `entropy`, and `topology_metrics` capture PSD aspect, entropy, and area/compactness/connectivity QA flags for Task Master stats + reporting.


In [ ]:
checklist = {
    "metrics_ready": True,
    "anchors_synced": True,
}
assert all(checklist.values())
checklist
